In [ ]:
# Python STL imports
import collections
from contextlib import redirect_stdout
import csv
import math
import os
import pydot
import random as rand
from random import sample
import timeit

# CNN framework imports
from keras import layers
from keras import backend as K
from keras.applications.imagenet_utils import preprocess_input
from keras.callbacks import EarlyStopping
from keras.datasets import fashion_mnist
from keras.initializers import glorot_uniform
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout
from keras.models import Model, Sequential
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils import plot_model
from keras.utils.data_utils import get_file
from keras.utils.np_utils import to_categorical
from keras.utils.vis_utils import model_to_dot

import numpy as np

from IPython.display import SVG

from tqdm import tqdm_notebook

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from sklearn.model_selection import train_test_split

In [ ]:
def _mnist_dataset(train_sample = 2000, test_sample = 500):
    """Helper to generate dataset for the tests
    
    Args:
        train_sample: how many samples for training set
        test_sample: how many samples for the test set
    
    Returns:
        Tuple t, where t[0] is the train set, t[1] is the validation set and t[2] is the test set
    """
    
    classes = 10
    width = 28
    height = 28
    channels = 1

    (X_train, Y_train), (X_test, Y_test)= fashion_mnist.load_data()

    K.set_image_data_format("channels_last")

    X_train = X_train.reshape(X_train.shape[0], width, height, channels) #.astype('float32') / 255 
    X_test = X_test.reshape(X_test.shape[0], width, height, channels) #.astype('float32') / 255
    Y_train = to_categorical(Y_train)
    Y_test = to_categorical(Y_test)
    X_train = X_train[:int(train_sample)]
    Y_train = Y_train[:int(train_sample)]
    X_test = X_test[:int(test_sample)]
    Y_test = Y_test[:int(test_sample)]

    X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1, shuffle= True)

    #dataset = ((X_train.astype("float32")/255, Y_train), (X_val.astype("float32")/255, Y_val), (X_test.astype("float32")/255, Y_test))
    dataset = ((X_train, Y_train), (X_val, Y_val), (X_test, Y_test))
    return dataset

In [ ]:
class Genome:
    """Class to manage constructing networks and constructing new sample networks.
    
    The structure is that all networks in this code are defined by a sequence. A sequence
    is a list of integers where each member is a choice. The index in sequences indicates what type of
    layer (dense, convolutional, pooling) and what choice in that layer is being described.
    
    Given a sequence, _Network() can produce a model.
    
    Please see |self.conv_block|, |self.dense_block|, and |self.pooling_block| to see the choices available
    per layer.
    
    Note: as of python3.7, the insertion order in a  dictionary is preserved. I.e. iterating over the keys
    below is guaranteed to iterate in the same order as the keys are declared on the dictionaries.
    """
    def __init__(self, max_conv=6, max_dense=2, max_filters =128, input_shape = (28,28,1),
                 classes =10, max_pooling= 0,  activation = 'relu',
                 max_filter_size =5, max_dense_units = 1024):
        """Initialize Genome class.
        
        Args:
            max_conv: the max number of convolutional layers
            max_dense: the max number of dense layers
            max_filters: the max number of filters
            input_shape: image input shape
            classes: the number of classes to classify
            max_pooling: the maximum number of pooling layers
            activation: activation method to pass to Activation layer
            max_filter_size: max size of filters
            max_dense_units: max number of dense units
        """
        self.max_conv = max_conv
        self.max_dense = max_dense
        self.max_pooling =max_pooling
        self.max_filters = max_filters 
        self.input_shape = input_shape
        self.classes = classes
        self.activation = activation
        self.max_filter_size = max_filter_size
        self.max_dense_units = max_dense_units
        self.history_fitness_sequences = []
        self.mutation_prob = 0.05
        
        #dictionaries with all possible parameters of CNN layers:
        self.conv_block = { 'on_off': [0,1],
                            'n_filters': [2**i for i in range(2, int(math.log(self.max_filters, 2))+1)],
                            'filter_sizes': [3 + 2*i for i in range(0, (max_filter_size-1)//2)],   
                            'batch_norm': [0,1],
                            'drop_out': [0, 0.2, 0.3, 0.4, 0.5]
                          }
        
        self.dense_block = { 'on_off': [0,1],
                             'units': [2**i for i in range(2, int(math.log(self.max_dense_units, 2))+1)] ,
                             'batch_norm': [0,1],
                             'drop_out': [0, 0.2, 0.3, 0.4, 0.5]
                           }
        
        self.pooling_block = {'on_off': [0,1]
                             }
        
        # genome sizes: 
        self.len_conv_block = len(self.conv_block) # number of genes assigned to conv_block
        self.len_dense_block = len(self.dense_block)  # number of genes assigned to dense_block
        self.len_pooling_block = len(self.pooling_block) # number of genes assigned to pooling_block
        
        #total length of genome sequence: 
        self.genome_length = self.max_conv * self.len_conv_block + self.max_dense *self.len_dense_block +\
                             self.max_conv *self.len_pooling_block
            
        #length of genome sequence assigned to convolutional_layer_blocks:     
        self.genome_conv_length = self.max_conv * self.len_conv_block
        
        #length of genome sequence assigned to dense_layer_blocks:
        self.genome_dense_length = self.max_dense *self.len_dense_block
        

    def _genome_initialization(self, method='random'):
        """Helper to initialize a genome.
        
        Args:
            method: str, random, min, or max. Depending on the choice, each choice in the
                    genome is either min, max, or random. See class description for choices.
        
        Returns:
            sequence, list of values for each genome choice to construct network
        """
        if method not in ['random', 'min', 'max']:
            raise Exception('Method %s unknown.' % method)
        if method == 'random':
            fn = np.random.choice
        elif method == 'min':
            fn = min
        elif method == 'max':
            fn = max
        individual=[] # empty list to store population 
        #first construct conv layers genes sequence
        for i in range(self.max_conv):
            for key in self.conv_block:
                individual.append(fn(self.conv_block[key]))
            for key in self.pooling_block:
                individual.append(fn(self.pooling_block[key]))

        #second constract dense layers genes sequence
        for i in range(self.max_dense):
            for key in self.dense_block:
                individual.append(fn(self.dense_block[key]))
        #assure that we have at least one conv layer:
        individual[0] = 1
        
        return individual
            
                     
    def _Network(self, individual):
        """Generate a network from |individual|.
        
        Args:
            individual: list of integers, template for each network later to construct a network.
        Returns:
            keras Model object constructed from template
        """
        
        X_input = Input(self.input_shape)
        X= X_input

        idx = 0
        for _ in range(self.max_conv):
            if individual[idx] ==1:
                n_filters = int(individual[idx+1])
                filter_size = int(individual[idx+2])
                X= Conv2D(n_filters, (filter_size, filter_size), padding ='same')(X)
                if individual[idx+3] ==1:  
                    X = BatchNormalization()(X)
                X = Activation(self.activation)(X)  
                if individual[idx+4] > 0 :  
                    X = Dropout(individual[idx+5])(X)
                
            idx += self.len_conv_block
            
            if individual[idx] ==1:
                if X.shape[1] > 7: 
                    X = MaxPooling2D((2, 2))(X)
            idx += self.len_pooling_block                      
            
        X = Flatten()(X)  
        
        for _ in range(self.max_dense):
            if individual[idx] ==1:
                X = Dense(int(individual[idx+1]), activation= self.activation)(X)
                
                if individual[idx+2] ==1:  
                    X = BatchNormalization()(X)
                X = Activation(self.activation)(X)   

                if individual[idx+3] > 0 :  
                    X = Dropout(individual[idx+3])(X)
                
            idx+= self.len_dense_block    
            
        X = Dense(self.classes, activation='softmax')(X)
        
        model = Model(inputs = X_input, outputs = X)
        
        return model
        
    
    def _crossover(self, sequences):
        """Randomly cross over two sequences.
        
        Randomly pick an index and create a new sequence out of all elements from the first sequence up
        to and excluding the index, and all members from the other sequences after that.
        
        Args:
            sequence: two member list of model template sequences
            
        Returns:
            list, model template sequence after crossing over the two sequences in |sequences|
        """
        
        cross_sequence =[]
        
        idx = rand.randint(0, len(sequences[0]))
        
        for i in range(idx):
            cross_sequence.append(sequences[0][i])
            
        for i in range(idx, len(sequences[0])):
            cross_sequence.append(sequences[1][i])
             
        #print(self.cross_sequence)
        return cross_sequence 
    
    def _mutate(self, sequence, num_mutations = None):
        """Mutate |sequence| |num_mutations| times.
        
        Args:
            individual: list of integers, template for each network later to construct a network
            num_mutations: number of mutations to perform, or None. If None, a random number
                           of mutations between 0 and 5 is chosen.
        
        Returns:
            mutated_sequence: list, model template sequence after mutations
        """
        
        if num_mutations == None:
            max_mutations = int(np.random.choice(5))
        else:
            max_mutations = int(num_mutations)
        #print('seq', sequence)    
        mutated_sequence = sequence
        for i in range(max_mutations):
            #print(i, max_mutations)
            #print(len(sequence) - 1)
            idx = int(np.random.choice(range(1, len(sequence) - 1)))
            if idx < self.max_conv*(self.len_pooling_block + self.len_conv_block):
                length = self.len_pooling_block + self.len_conv_block
                if idx%length < 5:
                    mutated_sequence[idx] = np.random.choice(self.conv_block[list(self.conv_block)[idx%length]])
                else:
                    mutated_sequence[idx] = np.random.choice(self.pooling_block['on_off'])

            else:
                idx = idx - self.max_conv*(self.len_pooling_block + self.len_conv_block)
                mutated_sequence[idx] = np.random.choice(self.conv_block[list(self.conv_block)[idx%4]])
                
        return mutated_sequence
    

In [ ]:
class Algorithm:
    """Class to run the algorithm and collect statistics."""
    
    def __init__(self, dataset, epochs=2, batch_size=200, verbose=1, iterations = 10,
                 population_size =10, sample_strategy = 'best'):
        """Initialize algorithm class.
        
        Args:
            dataset: dataset to use
            epochs: epochs to run over dataset
            batch_size: batch size for training and validation
            verbose: 0 indicates silent, 1 indicates progress bar, 2 indicates one line per epoch
                     (keras.models.Model)
            iterations: number of iterations to run the evolutionary algorithm for
            population_size: number of sequences in one population 
            sample_strategy: str, sampling strategy to find the next population from a previous population
        """
        
        (self.X_train, self.Y_train), (self.X_val, self.Y_val), (self.X_test, self.Y_test) = dataset
        self.epochs = epochs
        self.batch_size = batch_size
        self.verbose = verbose
        self.population_size = population_size
        self.iter_best_accuracy = []
        self.iter_best_time = []
        self.iter_best_sequence = []
        self.iter_best_params = []
        self.iterations = iterations
        self.mu_acc = []
        self.std_acc = []
        self.start = True
        self.full_sequence = []
        self.full_acc = []
        self.full_time = []
        self.full_params = []
        self. sample_strategy= sample_strategy
        
        
    def _individual_fitness(self, individual):
        """Evaluate sequence |individual|.
        
        Generate a Model from |indidividual|, train it, and evaluate it before reporting the results
        
        Args:
            individual: list, model template sequence
        
        Returns:
             tuple t, where t[0] is the sequence to generate the network
                            t[1] is the fitness accuracy(percentage)
                            t[2] is the time in seconds to evaluate the model
                            t[3] is the the number of parameters in the model
        """
        
        model = Genome()._Network(individual)  
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        es =  EarlyStopping(monitor= 'val_loss', patience=1)
        model.fit(self.X_train, self.Y_train, epochs = self.epochs, batch_size = self.batch_size, 
                  validation_data =(self.X_val, self.Y_val), callbacks=[es], verbose=self.verbose) 
        
        start = timeit.default_timer()
        preds = model.evaluate(self.X_test, self.Y_test)
        stop = timeit.default_timer()
        
        # This cleans out the tensorflow graph before creating a new one. As this code generates many models
        # and then discards them, this is critical to stay within memory bounds and not explode.
        # Needless to say: do not remove.
        K.clear_session()
              
        fitness_acc = preds[1]
        fitness_time = stop-start
        
        return individual, fitness_acc, fitness_time, model.count_params()
    
    def _population_fitness(self, population, best_history, full_history):
        """Given |population|, evaluate each member, and append to |best_history| and |full_history|.
        
        Args:
            population: list of sequences to evalute
            best_history: a tuple bh, where bh[0] is a list of the best accuracy per iteration
                                            bh[1] is a list of the runtimes per iteration
                                            bh[2] is a list of the number of params of the best performing network per iteration
                                            bh[3] is a list of actual sequences of highest performing networks per iteration
                                            bh[4] is the mean accuracy per iteration
                                            bh[5] is the standard deviation of the accuracy per iteration
                          best_history can be though of a matrix as well.
            full_history: a tuple fh, where fh[0] is a list of the accuracy per experiment (iterations * population size)
                                            fh[1] is a list of the runtimes per experiment (iterations * population size) 
                                            fh[2] is a list of the number of params per experiment (iterations * population size)
                                            fh[3] is a list of the sequences for each experiment
                                            
        Returns:
            A tuple t, where t[0] is the a fitness tuple for |population|
                                t[0][0] contains a list of accuracy per experiment
                                t[0][1] contains a list of validation time per experiment
                                t[0][2] contains a list of number of parameters per experiment
                                t[0][3] contains a list of sequences samples
                              t[1] is |best_history| expanded with the best result from |population|
                              t[2] is |full_history| expanded with all the results from |population| 

        """
        
        self.fitness_sequence = []
        self.fitness_acc = []
        self.fitness_time = []
        self.fitness_params = []
        self.mu_acc = [] 
        self.std_acc = []
        
        (self.iter_best_accuracy, self.iter_best_time, self.iter_best_params, self.iter_best_sequence, self.mu_acc, self.std_acc) = best_history
        (self.full_acc, self.full_time, self.full_params, self.full_sequence) = full_history
        
        for individual in tqdm_notebook(population):
            
            individual, fitness_acc, fitness_time, fitness_params  = self._individual_fitness(individual)
            
            self.fitness_acc.append(fitness_acc)
            self.fitness_time.append(fitness_time)
            self.fitness_params.append(fitness_params)
            self.fitness_sequence.append(individual)
  
            
        fitness = (self.fitness_acc, self.fitness_time, self.fitness_params, self.fitness_sequence)
        
        for full, run in [(self.full_sequence, self.fitness_sequence),
                          (self.full_acc, self.fitness_acc),
                          (self.full_time, self.fitness_time),
                          (self.full_params, self.fitness_params)]:
            full.extend(run)
            
        full_history =  (self.full_acc, self.full_time, self.full_params, self.full_sequence)
        

        idx = self.fitness_acc.index(max(self.fitness_acc))
        self.iter_best_accuracy.append(self.fitness_acc[idx])
        self.iter_best_time.append(self.fitness_time[idx])
        self.iter_best_params.append(self.fitness_params[idx])
        self.iter_best_sequence.append(self.fitness_sequence[idx])
        self.mu_acc.append(np.mean(self.fitness_acc))
        self.std_acc.append(np.std(self.fitness_acc))
            
        best_history = (self.iter_best_accuracy, self.iter_best_time, self.iter_best_params, self.iter_best_sequence, self.mu_acc, self.std_acc)
        
        return fitness, best_history, full_history

    def _main(self, initial_population, min_time, max_time):
        """Main function to run the algorithm.
        
        Will run algorithm for |self.iterations| iterations before returning all information about
        the run.
        
        Args:
            initial_population: list of sequences to start the experiment with
            min_time: prediction time for a minimal (smallest) network on given hardware
            max_time: prediction time on maximal (biggest) network on given hardware
            
        Returns:
            tuple t, where t[0] is best_history, matrix // see _population_fitness() for details on shape and elements
                           t[1] is best_accuracy, accuracy of highest accuracy sequence
                           t[2] is best_run_time, validation time of highest accuracy sequence
                           t[3] is best_params, number of parameters for highest accuracy sequence
                           t[4] is the keras.models.Model object for the best performing sequence
                           t[5] is the full_history, matrix // see _population_fitness() for details on shape and elements
                           t[6] is the total run time for the algorithm
        """
        
        print("Iteration:", 0)
        
        best_history = (self.iter_best_accuracy, self.iter_best_time, self.iter_best_params, self.iter_best_sequence, self.mu_acc, self.std_acc)
        full_history= (self.full_acc, self.full_time, self.full_params, self.full_sequence)
        
        self.population = initial_population
        
        fitness, best_history, full_history = self._population_fitness(self.population, best_history, full_history)
        
        start = timeit.default_timer()
        for i in tqdm_notebook(range(1, self.iterations)):
            print("Iteration:", i)
            self.population = Population(self.population_size)._offspring(fitness, best_history, full_history, self.sample_strategy, min_time, max_time )
            fitness, best_history, full_history = self._population_fitness(self.population, best_history, full_history)
            #print(best_history)
            # Turn fitness_time into an array to extract information
            time_array = np.array(fitness[1])
            global_time_stats[self.sample_strategy].append((time_array.mean(), time_array.std()))
            
        stop = timeit.default_timer()
        best_accuracy, best_run_time, best_params, best_model =  Population(self.population_size)._get_final_statistics(best_history)
        total_run_time = (stop-start)/60
        
        return best_history, best_accuracy, best_run_time, best_params, best_model, full_history, total_run_time

In [1]:
class Population:
    """Class to handle populations.
    
    It provides tools to generate random populations, pick a new population given a previous (or all previous)
    population(s), and get final statistics for the best sequence found throughout the algorithm.
    """
    
    # Default weights for fitness functions considering more than just accuracy.
    # Please look at _offspring() below to see the fitness functions
    w_accuracy = 1
    w_time = -1
    w_params = -1
    
    def __init__(self, size):
        """Initialize the population class.
        
        Args:
            size: number of sequences in one population
        """
        
        self.size = size
        self.population = []
        self.prob_crossover = 0.5
        self.prob_mutation = 0.5
        self.iter_best_accuracy = []
        self.iter_run_time = []
        self.iter_best_sequences = []
        self.mu_acc = []
        self.std_acc = []
        
    @classmethod
    def set_weights(cls, w_accuracy = 1, w_time = -1, w_params = -1):
        """Helper method for weights.
        
        Below in fitness functions you will find that |w_accuracy|, |w_time| and |w_params| are 
        used to decide how important each of those elements is for a fitness. This class function
        sets those weights to experiment with bigger or smaller impacts.
        
        Note: not all fitness functions use all parameters, please take a look below in _offspring()
        
        Args:
            w_accuracy: weight that the accuracy of a sequence should have on fitness functions
            w_time: weight that the predict time should have fitness functions
            w_params: weight that log of number of params should have on fitness functions
        """
        cls.w_accuracy = w_accuracy
        cls.w_time = w_time
        cls.w_params = w_params
        
    def _population_initialization(self):
        """Append random sequences to |self.population| for |self.size| iterations for initial population."""
        while len(self.population) < self.size:
            self.population.append(Genome()._genome_initialization())
    
        return self.population
    
    
    def _offspring(self, fitness, best_history, full_history, sample_strategy = 'random', min_time=0, max_time=1):
        """Generate an offsprign population.
        
        Note on gibbs functions: In order to facilitate debugging, all gibbs based strategies will append
        to a file called results/[experiment_name]_[sample_strategy]_weights.txt.
        Every entry in that file has the list of gibbs weights and full accuraries.
        
        
        Args:
            fitness: matrix, // see _population_fitness() for details on shape and elements
            best_history: matrix // see _population_fitness() for details on shape and elements 
            full_history: matrix // see _population_fitness() for details on shape and elements
            sample_strategy: a valid sampling strategy, currently supported strategies are:
                             random
                             tournament
                             tournament_epsilon
                             gibbs
                             gibbs_epsilon
                             tournament_gibbs
                             tournament_fit2
                             tournament_epsilon_fit2
                             tournament_gibbs_epsilon_fit2
                             tournament_fit2(norm)
                             tournament_epsilon_fit2(norm)
                             tournament_gibbs_epsilon_fit2(norm)
                             tournament_fit3 # is not used in this poject and is provided as example for future research
                             // see below in the code for details on each strategy. There are comments
            min_time: validation time on a minimal network on running hardware, used for normalization
            max_time: validation time on a maximal network on running hardware, used for normalization
        
        Returns:
            self.offspring: a new list of sequences
        """
        
        self.offspring = []
        
        (self.fitness_acc, self.fitness_time, self.fitness_params, self.fitness_sequences) = fitness
        (self.iter_best_accuracy, self.iter_run_time, self.iter_best_params, self.iter_best_sequence, self.mu_acc, self.std_acc) = best_history
        
        
        #get two best individuals from population fitness:
        top2_acc, top2_run_time, top2_params, top2_sequence = self._best_individuals()
        
        best_acc = max(top2_acc)
        idx = top2_acc.index(best_acc)
        best_run_time = top2_run_time[idx]
        best_sequence = top2_sequence[idx]
        best_params = top2_params[idx]
        mu_acc = np.mean(self.fitness_acc)
        std_acc = np.std(self.fitness_acc)

        print("Best accuracy for iteration: {0:3f}, with prediction time: {1:3f}".format(best_acc,best_run_time))
        print("Mean accuracy for iteration: {0:3f}, std: {1:3f}".format(mu_acc, std_acc))      
        
        # Used below in the gibbs methods.
        self.full_acc, self.full_time, self.full_params, self.full_sequence = full_history
        
        if sample_strategy == 'random':
            # Randomly pick all elements for the new population.
            for _ in range(self.size):
                self.offspring.append(Genome()._genome_initialization()) 
    
        elif sample_strategy == 'tournament':   
            #cross two best models:
            self.offspring.append(Genome()._crossover(top2_sequence))  
            
            for _ in range(self.size-1):
                sequences = []
                # randomly choose 2 models, then chosse best from this two models and append it to population. 
                # Repeat two times                   
                for _ in range(2):
                    sequences.append(self._best_individual(self._random_choice()))  
                    #cross two chosen models: 
                self.offspring.append(Genome()._crossover(sequences))
                
            # randomly mutate
            for i in range(1, len(self.offspring)):
                self.offspring[i] = Genome()._mutate(self.offspring[i], num_mutations = 3) 
        
            
        if sample_strategy == 'tournament_epsilon':   
            
            #cross two best models:
            self.offspring.append(Genome()._crossover(top2_sequence))  
            
            for _ in range(self.size-3):
                sequences = []
                # randomly choose 2 models, then chosse best from this two models and append it to population. 
                # Repeat two times                   
                for _ in range(2):
                    sequences.append(self._best_individual(self._random_choice()))  
                    #cross two chosen models: 
                self.offspring.append(Genome()._crossover(sequences))

            #add two fully mutated sequence: 
            for _ in range(2):
                self.offspring.append(Genome()._genome_initialization()) 
                
            # randomly mutate
            for i in range(1, len(self.offspring)):
                self.offspring[i] = Genome()._mutate(self.offspring[i], num_mutations = 3)        
        

        elif sample_strategy == 'gibbs':
            # Choose sequences from previously seen sequences in a sampling fashion, using their
            # normalized accuracy as a probability of being chosen.
            self.gibbs_weights = [i/sum(self.full_acc) for i in self.full_acc]

            for _ in range(self.size):
                self.offspring.append(self._gibbs_sampling()) 

            # randomly mutate
            for i in range(len(self.offspring)):
                self.offspring[i] = Genome()._mutate(self.offspring[i], num_mutations = 3)   
                
                
        elif sample_strategy == 'gibbs_epsilon':   
            # Choose all but 2 sequences from previously seen sequences in a sampling fashion, using their
            # normalized accuracy as a probability of being chosen.
            # Choose the remaining two sequences randomly.
            self.gibbs_weights = [i/sum(self.full_acc) for i in self.full_acc]

            for _ in range(self.size-2):
                self.offspring.append(self._gibbs_sampling()) 
                
            #add two fully mutated sequence: 
            for _ in range(2):
                self.offspring.append(Genome()._genome_initialization()) 
                
            # randomly mutate
            for i in range(len(self.offspring)):
                self.offspring[i] = Genome()._mutate(self.offspring[i], num_mutations = 3)   
                
        elif sample_strategy == 'tournament_gibbs': 
            # Choose sequences from previously seen sequences in a sampling fashion, using their
            # normalized accuracy as a probability of being chosen.
            # For each output sequence, choose two sequences as described above and cross them
            self.gibbs_weights = [i/sum(self.full_acc) for i in self.full_acc]

            for _ in range(self.size):
                sequences = []
                for _ in range(2):
                    sequences.append(self._gibbs_sampling()) 
                #cross two chosen models:
                self.offspring.append(Genome()._crossover(sequences))
            # randomly mutate
            for i in range(len(self.offspring)):
                self.offspring[i] = Genome()._mutate(self.offspring[i], num_mutations = 3)          
        
        elif sample_strategy =='tournament_fit2':
            # Choose top 2 sequences using fit2 (accuracy and validation time) as fitness function
            self.fit = [(Population.w_accuracy * acc + (Population.w_time * time))
                        for acc, time in zip(self.fitness_acc, self.fitness_time)]
        
            print('self.fit', self.fit)
            
            top2_fit = sorted(self.fit)[-2:]
            
            top2_sequence = [self.fitness_sequences[self.fit.index(f)] for f in top2_fit]
            
            #print(idx, self.best_accuracy, self.best_sequence)
            self.offspring.append(Genome()._mutate(self.fitness_sequences[idx], num_mutations = 2))  

            #cross two best models:
            self.offspring.append(Genome()._crossover(top2_sequence))  
            
            for _ in range(self.size-1):
                sequences = []
                # randomly choose 2 models, then chosse best from this two models and append it to population. 
                # Repeat two times                   
                for _ in range(2):
                    sequences.append(self._best_individual(self._random_choice(metric = 'fit')))  
                    #cross two chosen models: 
                self.offspring.append(Genome()._crossover(sequences))  

            # randomly mutate
            for i in range(1, len(self.offspring)):
                self.offspring[i] = Genome()._mutate(self.offspring[i], num_mutations = 3) 
        
        
        elif sample_strategy=='tournament_epsilon_fit2':
            # Choose top 2 sequences using fit2 (accuracy and validation time) as fitness function
            # Add two fully random sequences.
            self.fit = [(Population.w_accuracy * acc + (Population.w_time*time))
                        for acc, time in zip(self.fitness_acc, self.fitness_time)]
        
            top2_fit = sorted(self.fit)[-2:]
            
            top2_sequence = [self.fitness_sequences[self.fit.index(f)] for f in top2_fit]
            
            #print(idx, self.best_accuracy, self.best_sequence)
            self.offspring.append(Genome()._mutate(self.fitness_sequences[idx], num_mutations = 2))  

            #cross two best models:
            self.offspring.append(Genome()._crossover(top2_sequence))  
            
            for _ in range(self.size-3):
                sequences = []
                # randomly choose 2 models, then chosse best from this two models and append it to population. 
                # Repeat two times                   
                for _ in range(2):
                    sequences.append(self._best_individual(self._random_choice(metric = 'fit')))  
                    #cross two chosen models: 
                self.offspring.append(Genome()._crossover(sequences))  
            
            #add two fully mutated sequence: 
            for _ in range(2):
                self.offspring.append(Genome()._genome_initialization())   
            
            # randomly mutate
            for i in range(1, len(self.offspring)):
                self.offspring[i] = Genome()._mutate(self.offspring[i], num_mutations = 3)
                
        elif sample_strategy == 'tournament_gibbs_epsilon_fit2':  
            # Choose sequences from previously seen sequences in a sampling fashion, using their
            # normalized fit2 (accuracy and validation time) as a probability of being chosen.
            # For each but 2 output sequences, choose two sequences as described above and cross them
            # Pick 2 random output sequences
            self.gibbs_weights = [max((Population.w_accuracy * acc +  (time* Population.w_time)),0)
                        for acc, time in zip(self.full_acc, self.full_time)]
            self.gibbs_weights = [i/sum(self.gibbs_weights) for i in self.gibbs_weights]
            
            for _ in range(self.size-2):
                sequences = []
                for _ in range(2):
                    sequences.append(self._gibbs_sampling()) 
                #cross two chosen models:
                self.offspring.append(Genome()._crossover(sequences))
                
            #add two fully mutated sequence: 
            for _ in range(2):
                self.offspring.append(Genome()._genome_initialization())     
                
            # randomly mutate
            for i in range(len(self.offspring)):
                self.offspring[i] = Genome()._mutate(self.offspring[i], num_mutations = 3) 


        elif sample_strategy=='tournament_fit2(norm)':
            # Choose top 2 sequences using fit2 (accuracy and validation time) as fitness function
            # The validation_time is normalized against |min_time| and |max_time|.
            self.fit = [(Population.w_accuracy * acc + (Population.w_time*(time-min_time)/(max_time-min_time)))
                        for acc, time in zip(self.fitness_acc, self.fitness_time)]
            
            print('self.fit', self.fit)
        
            top2_fit = sorted(self.fit)[-2:]
            
            top2_sequence = [self.fitness_sequences[self.fit.index(f)] for f in top2_fit]
            
            #print(idx, self.best_accuracy, self.best_sequence)
            self.offspring.append(Genome()._mutate(self.fitness_sequences[idx], num_mutations = 2))  

            #cross two best models:
            self.offspring.append(Genome()._crossover(top2_sequence))  
            
            for _ in range(self.size-1):
                sequences = []
                # randomly choose 2 models, then chosse best from this two models and append it to population. 
                # Repeat two times                   
                for _ in range(2):
                    sequences.append(self._best_individual(self._random_choice(metric = 'fit')))  
                    #cross two chosen models: 
                self.offspring.append(Genome()._crossover(sequences))  

            # randomly mutate
            for i in range(1, len(self.offspring)):
                self.offspring[i] = Genome()._mutate(self.offspring[i], num_mutations = 3)
        
        elif sample_strategy=='tournament_epsilon_fit2(norm)':
            # Choose top 2 sequences using fit2 (accuracy and validation time) as fitness function
            # The validation_time is normalized against |min_time| and |max_time|.
            # Add two fully random sequences.
            self.fit = [(Population.w_accuracy * acc + (Population.w_time*(time-min_time)/(max_time-min_time)))
                        for acc, time in zip(self.fitness_acc, self.fitness_time)]
        
            top2_fit = sorted(self.fit)[-2:]
            
            top2_sequence = [self.fitness_sequences[self.fit.index(f)] for f in top2_fit]
            
            #print(idx, self.best_accuracy, self.best_sequence)
            self.offspring.append(Genome()._mutate(self.fitness_sequences[idx], num_mutations = 2))  

            #cross two best models:
            self.offspring.append(Genome()._crossover(top2_sequence))  
            
            for _ in range(self.size-3):
                sequences = []
                # randomly choose 2 models, then chosse best from this two models and append it to population. 
                # Repeat two times                   
                for _ in range(2):
                    sequences.append(self._best_individual(self._random_choice(metric = 'fit')))  
                    #cross two chosen models: 
                self.offspring.append(Genome()._crossover(sequences))  
            
            #add two fully mutated sequence: 
            for _ in range(2):
                self.offspring.append(Genome()._genome_initialization())   
            
            # randomly mutate
            for i in range(1, len(self.offspring)):
                self.offspring[i] = Genome()._mutate(self.offspring[i], num_mutations = 3)
                
        elif sample_strategy == 'tournament_gibbs_epsilon_fit2(norm)':  
            # Choose sequences from previously seen sequences in a sampling fashion, using their
            # normalized fit2 (accuracy and validation time) as a probability of being chosen.
            # The validation_time is normalized against |min_time| and |max_time|.
            # For each but 2 output sequences, choose two sequences as described above and cross them
            # Pick 2 random output sequences
            self.gibbs_weights = [max((Population.w_accuracy * acc + ((time- min_time)/(max_time-min_time) * Population.w_time)),0)
                        for acc, time in zip(self.full_acc, self.full_time)]
            self.gibbs_weights = [i/sum(self.gibbs_weights) for i in self.gibbs_weights]
            
            for _ in range(self.size-2):
                sequences = []
                for _ in range(2):
                    sequences.append(self._gibbs_sampling()) 
                #cross two chosen models:
                self.offspring.append(Genome()._crossover(sequences))
                
            #add two fully mutated sequence: 
            for _ in range(2):
                self.offspring.append(Genome()._genome_initialization())     
                
            # randomly mutate
            for i in range(len(self.offspring)):
                self.offspring[i] = Genome()._mutate(self.offspring[i], num_mutations = 3)      
                
        elif sample_strategy =='tournament_fit3':
            # Choose top 2 sequences using fit3 (accuracy and number of parameters) as fitness function
            self.fit = [(Population.w_accuracy * acc + (Population.w_params * param))
                        for acc, param in zip(self.fitness_acc, self.fitness_params)]
        
            print('self.fit', self.fit)
            
            top2_fit = sorted(self.fit)[-2:]
            
            top2_sequence = [self.fitness_sequences[self.fit.index(f)] for f in top2_fit]
            
            #print(idx, self.best_accuracy, self.best_sequence)
            self.offspring.append(Genome()._mutate(self.fitness_sequences[idx], num_mutations = 2))  

            #cross two best models:
            self.offspring.append(Genome()._crossover(top2_sequence))  
            
            for _ in range(self.size-1):
                sequences = []
                # randomly choose 2 models, then chosse best from this two models and append it to population. 
                # Repeat two times                   
                for _ in range(2):
                    sequences.append(self._best_individual(self._random_choice(metric = 'fit')))  
                    #cross two chosen models: 
                self.offspring.append(Genome()._crossover(sequences))  

            # randomly mutate
            for i in range(1, len(self.offspring)):
                self.offspring[i] = Genome()._mutate(self.offspring[i], num_mutations = 3)          
                
        
        if 'gibbs' in sample_strategy:
            weight_f = 'results/%s_%s_weights.txt' % (experiment_name, str(sample_strategy))
            with open(weight_f, 'a') as f:
                f.write('full_acc:\n %s\n' % str(self.full_acc))
                f.write('gibbs_weights:\n %s\n' % str(self.gibbs_weights))
                f.write('-----------------\n')
                
        return self.offspring
          
    def _best_individuals(self):
        """Find information about the best two sequences in a set.
        
        Returns: tuple t, where t[0] contains the best accuracy
                                t[1] contains the runtime of that sequence
                                t[2] contains the number of parameters of that sequence
                                t[3] contains the actual sequence
                 each entry has two members, and they are in sync, so t[0][1] belongs with t[1][1]
        """
        run_time = []
        parameters = []
        best_sequences = []
        best_accuracy = sorted(self.fitness_acc)[-2:] 
        for accuracy in best_accuracy:
            idx = self.fitness_acc.index(accuracy)                   
            run_time.append(self.fitness_time[idx])
            parameters.append(self.fitness_params[idx])
            best_sequences.append(self.fitness_sequences[idx])
        
        return best_accuracy, run_time, parameters, best_sequences                               
    
    def _random_choice(self, metric = 'acc'):
        """Randomly choose two sequences.
        
        Args:
            metric: one of 'acc' or 'fit', which metric to report about the random sequence chosen to caller
        
        Returns:
            tuple t, where t[0] is a list of sequences and t[1] is the corresponding fitness metric
                t is always size 2
        """
        sequences = [] 
        fit = []
        for _ in range(2):                       
            idx = int(rand.uniform(0, self.size - 1))
            sequences.append(self.fitness_sequences[idx])
            
            if metric == 'acc':
                fit.append(self.fitness_acc[idx])
            elif metric == 'fit':
                fit.append(self.fit[idx])
                
        return (sequences, fit)
    
    
    def _gibbs_sampling(self):
        """Sample one sequence using |self.gibbs_weights|.
        
        Note: make sure that the code calling this properly initialized |self.gibbs_weights|
        
        Returns:
            a sequence from |self.full_sequence|
        """
        idx = int(np.random.choice(range(0, len(self.full_acc)), 1, p = self.gibbs_weights))
        gibbs_sequence = self.full_sequence[idx]

        return gibbs_sequence
    
                               
    def _best_individual(self, rand_sequences):
        """Find the highest accuracy sequence in |rand_sequences|.
        
        Args:
            rand_sequences: list of sequences
            
        Returns:
            best_sequence, sequence in |rand_sequences| with highest accuracy
        """
        (sequences, accuracy) = rand_sequences
        idx = accuracy.index(max(accuracy))
        best_sequence = sequences[idx]
        
        return best_sequence     
    
    def _get_final_statistics(self, best_history):
        """Extract the statistics out of |best_history|.
        
        Args:
            best_history: tuple of tuples, please see population_fitness() for shape and meaning
        
        Returns:
            tuple t, where t[0] is the best accuracy overall a sequence produced
                           t[1] is the runtime of that sequence
                           t[2] is the number of parameters of that sequence
                           t[3] is the keras.models.Model object corresponding to that best sequence
        
        """
        
        (self.iter_best_accuracy, self.iter_run_time, self.iter_best_params, self.iter_best_sequences, self.mu_acc, self.std_acc) = best_history
        best_accuracy = max(self.iter_best_accuracy)
        idx =  self.iter_best_accuracy.index(best_accuracy)    
        best_sequence = self.iter_best_sequences[idx]
        best_run_time = self.iter_run_time[idx]
        best_params = self.iter_best_params[idx]
        best_model = Genome()._Network(best_sequence)
        
        return best_accuracy, best_run_time, best_params, best_model

In [ ]:
def _initialize_population(experiment_name = '', population_size = 10):
    """Helper to initialize population.
    
    For some experiments one wants to reuse the same initial population and for others not.
    If a population file already exists with experiment_name, the population is loaded,
    otherwise a new population for the experiment is generated and cached in the experiment's
    initial population file.
    
    Args:
        experiment_name: name of the experiment population
        population_size: number of sequences in a population
    
    Returns:
        initial_population: list of sequences, either from file or newly generated
    """
    
    population_f = os.path.join('results', '%s_initial.csv' % experiment_name)
    
    if os.path.exists(population_f):
        print('Reusing existing population %s' % experiment_name)
        return _get_initial_population(population_f)
    print('Population %s not defined yet. Creating it now' % experiment_name)
    
    # No luck, the population doesn't exist yet.
    initial_population = Population(population_size)._population_initialization()

    # Cache the population for future use here.
    with open(population_f, 'w') as f:
        wr = csv.writer(f, delimiter=',')
        wr.writerows(initial_population)     
                 
    return initial_population       

In [ ]:
def _get_initial_population(population_file):
    """Load caches population from |population_file|.
    
    Note: This assumes the caller checked that the file exists.
    
    Args:
        population_file: filename of the population csv file
    
    Returns:
        initial_population, list of sequences found at |population_file|
    """
    with open(population_file, 'r') as f:
        reader = csv.reader(f, delimiter=',')
        initial_population = list(reader)

    for i in range(len(initial_population)):
        for j in range(len(initial_population[0])):
            initial_population[i][j] = float(initial_population[i][j])       
            
            
    return initial_population

In [ ]:
def _run(initial_population , dataset, epochs = 15, 
         batch_size = 200, verbose= 0, iterations = 20,population_size = 10, 
         sample_strategy ='random', experiment_name = '5', only_time_bound=False):
    """Run the CNN search.
    
    Args:
        initial_population: list of sequences to serve as initial population
        dataset: dataset to use for search
        epochs: epochs to run over dataset
        batch_size: batch size for training and validation
        verbose: 0 indicates silent, 1 indicates progress bar, 2 indicates one line per epoch
                 (keras.models.Model)
        iterations: number of iterations to run the evolutionary algorithm for
        population_size: number of sequences in one population
        sample_strategy: str, sampling strategy to find the next population from a previous population
        experiment_name: name for the experiment, to tag output files with
        only_time_bound: if true, only the validation time for largest and smallest network will print before
                         exiting (experimental, to see validation time spread on given hardware)
    
    """
    
    algorithm = Algorithm(dataset=dataset, epochs=epochs,\
                          batch_size=batch_size, 
                          verbose= verbose, \
                          iterations = iterations, \
                          population_size = population_size, \
                          sample_strategy = sample_strategy)
    
    
    max_res = algorithm._individual_fitness(Genome()._genome_initialization(method='max'))
    min_res  = algorithm._individual_fitness(Genome()._genome_initialization(method='min'))
    max_acc, max_time = max_res[1], max_res[2]
    min_acc, min_time = min_res[1], min_res[2]
    
    print('The min network predict accuracy: {0:3f}, with prediction time: {1:3f}'.format(min_acc, min_time))
    print('The max network predict accuracy: {0:3f}, with prediction time: {1:3f}'.format(max_acc, max_time))
    if only_time_bound:
        return min_time, max_time

    best_history_top2, best_accuracy_top2, best_run_time_top2, best_params_top2, best_model_top2,full_history_top2, total_run_time_top2= algorithm._main(initial_population, min_time, max_time )
    accuracy_top2, run_time_top2, params_top2, sequence_top2, mu_acc_top2, std_acc_top2 = best_history_top2
    full_acc_top2, full_time_top2, full_params_top2, full_sequence_top2= full_history_top2

    # Note: all files that this outputs are in results, with this base template, meaning that outputs
    # are tagged by sample strategy and experiment name.
    results_base_template = 'results/%s_%s' % (experiment_name, str(sample_strategy))
    name_summary = '%s_%s' % (results_base_template, 'summary.txt')
    # Print and store the summary text. The summary text is pretty much just a summary of the best error,
    # and prediction time, total train time, and a summary of the model that produced those values.
    with open(name_summary, 'w') as f:
        with redirect_stdout(f):
            print("Best error: {0:3f}, with prediction time: {1:3f}".format(best_accuracy_top2,best_run_time_top2))
            print("Total train time: {0:2f}".format(total_run_time_top2), "min")
            print("Best model:", best_model_top2.summary())
    with open(name_summary, 'r') as f:
        print(f.read())

    # Plot accuracy over iterations
    plt.ylabel('Test Accuracy')
    plt.xlabel('Iteration')
    plt.title('Evolution Startegy with '+str(sample_strategy)+' Sampling')    
    plt.errorbar(range(iterations), mu_acc_top2, std_acc_top2,  marker='^')
    plt.plot(range(iterations), accuracy_top2)
    plt.legend(['Best accuracy', 'Mean statistic'])
    plt.grid()
    name_plot = '%s.%s' % (results_base_template, 'svg')
    plt.savefig(name_plot, format='svg')
    plt.show()
    
    # Dump results into a csv to make analysis with other tools easier later
    name_res = '%s_%s'% (results_base_template, 'results.csv')
    # Save model as a .h5 file to reuse later if desired
    name_model = '%s.%s' % (results_base_template, 'h5')
    
    top2 = (best_history_top2, full_history_top2, [best_accuracy_top2, best_run_time_top2, best_params_top2, total_run_time_top2])
    with open(name_res,"w") as f:
        wr = csv.writer(f, delimiter=',')
        wr.writerows(top2)

    best_model_top2.save(name_model) 

In [ ]:
# List of parameters to change manually or have a config file overwrite using papermill.
# Essentially, these parameters will be passed into _run() below, and allow for one cell to control
# different experiments in the same notebook.
# The papermill module inputs a cell below this cell with overwrites based on the configuration file
# provided so that parametrized experiments can be run.

# Note that experiment_name's first 4 characters are used to define the initial population. So
# a set of experiments that wants to share the same initial population should all start with
# the same first four characters, while a two experiments that wish to have (likely) distinct
# initial populations should have distinct first 4 characters.

# ---- Again, summarizing ---- 
# If you wish to run this in an interactive notebook, this is the place to change what experiment
# to run, and rerunning this cell before rerunning _run() below is a valid way of running multiple experiments
# 
# If you wish to run this with paramRunner.py, the values here are the default and will be overwritten
# by any of them if they are defined in the config .json file supplied to paramRunner.py
#
# Familiarize yourself with the impact and valid values for each of these values in the classes
# Algorithm, Genome, and Population before making modifications, and take a look at
# _run() and _initial_population to see the impact of experiment_name

experiment_name = 's1_3'
population_size = 15
sample_strategy = 'tournament_epsilon'
epochs = 15
iterations = 20
batch_size = 200
train_sample = 60000
test_sample = 10000
w_accuracy = 1
w_time = -1
w_params = -1  #not used in this project and is provided for future reseach and model extension
dry_run = False
only_time_bound = False # Whether to calculate the time bounds for min and max network only

In [ ]:
# Load the dataset into memory in its own cell so that if so desired the algorithm can be
# rerun below without needing to reload the dataset.
dataset = _mnist_dataset(train_sample = train_sample, test_sample = test_sample)

In [ ]:
print('Would have run with')
print('experiment-name: ', experiment_name)
print('population-size: ', population_size)
print('sample-strategy: ', sample_strategy)
print('epochs: ', epochs)
print('iterations: ', iterations)
print('batch-size: ', batch_size)
print('train-sample: ', train_sample)
print('test-size: ', test_sample)

In [ ]:
initial_population = _initialize_population(experiment_name = experiment_name[:4], population_size = population_size)
# Initialize weights for the prediction function properly
Population.set_weights(w_accuracy=w_accuracy, w_time=w_time, w_params=w_params)

In [ ]:
# Time stats. This dictionary contains time statistics. More precisely: the key is the sampling strategy. The output is
# a list where each index is the iteration. The content is a tuple. element[0] is the mean time. element[1] is the time std
global_time_stats = collections.defaultdict(list)

In [ ]:
if dry_run:
    print('Just a dry-run to see config and generate population. Bye')
else:
    # Initialize folder to dump results
    if not os.path.exists('results'):
        os.mkdir('results')
    _run(initial_population, dataset, sample_strategy = sample_strategy, experiment_name = experiment_name,
         epochs = epochs, iterations = iterations, population_size = population_size, batch_size = batch_size,
         only_time_bound=only_time_bound)